In [8]:
import pandas as pd
from datetime import datetime
import pytz

time_zone = pytz.timezone('Asia/Kolkata')
current_time= datetime.now(time_zone)

In [9]:
df_secondary = pd.read_csv('ykb_secondary_pending_abhi.csv')


In [10]:
df_secondary_zone_wise  = df_secondary.groupby('bag_type_ph').size()
print(df_secondary_zone_wise)

bag_type_ph
B1              1
B2              2
B3             49
B4              9
B5           6144
B6             65
D0LYKB         27
MR              2
NE              2
ZO           1629
ZOJFELYKB       1
dtype: int64


In [11]:
df_secondary_zo = df_secondary[df_secondary['bag_type_ph'] == "ZO"]
df_secondary_zo['bag_facility_source_name'].fillna("Not Found" , inplace=True)
df_secondary_zo_ph  = df_secondary_zo[df_secondary_zo['bag_facility_source_name'].str.contains("DEL_PL|Bhiwadi|Bhiwani|Bilaspur")]
df_secondary_zo_sph = df_secondary_zo[~df_secondary_zo.index.isin(df_secondary_zo_ph.index)]

df_secondary_zo_ph1 = df_secondary_zo_ph.groupby('bag_facility_source_name').size()
df_secondary_zo_sph1 = df_secondary_zo_sph.groupby('bag_facility_source_name').size()

print(f"Secondary Pending ZO PH: {sum(df_secondary_zo_ph1)}")
print(f"Secondary Pending ZO SPH: {sum(df_secondary_zo_sph1)}")
print(f"Secondary Total Pending ZO: {sum(df_secondary_zo_ph1) + sum(df_secondary_zo_sph1)} ")


Secondary Pending ZO PH: 1396
Secondary Pending ZO SPH: 233
Secondary Total Pending ZO: 1629 


/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/200542944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_secondary_zo['bag_facility_source_name'].fillna("Not Found" , inplace=True)


In [12]:
df_secondary_b5 = df_secondary[df_secondary['bag_type_ph'] == "B5"]
df_secondary_b5['bag_facility_source_name'].fillna("Not Found" , inplace=True)

df_secondary_b5_ph = df_secondary_b5[df_secondary_b5['bag_facility_source_name'].str.contains("DEL_PL|BHiwadi|Bhiwani|Bilaspur")]
df_secondary_b5_sph = df_secondary_b5[~df_secondary_b5.index.isin(df_secondary_b5_ph.index)]

df_secondary_b5_ph1 = df_secondary_b5_ph.groupby('bag_facility_source_name').size()
df_secondary_b5_sph1 = df_secondary_b5_sph.groupby('bag_facility_source_name').size()

print(f"Secondary Pending B5 PH: {sum(df_secondary_b5_ph1)}")
print(f"Secondary Pending B5 SPH: {sum(df_secondary_b5_sph1)}")
print(f"Secondary Total Pending B5: {sum(df_secondary_b5_ph1) + sum(df_secondary_b5_sph1)} ")

Secondary Pending B5 PH: 5780
Secondary Pending B5 SPH: 364
Secondary Total Pending B5: 6144 


/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/180343183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_secondary_b5['bag_facility_source_name'].fillna("Not Found" , inplace=True)


In [13]:
def calculate_and_categorize_time(dataframe, column_name, current_time  , bucket_name):
    df_time_wise = pd.to_datetime(dataframe[column_name])
    df_hour_wise = (current_time - df_time_wise) / pd.Timedelta(hours=1)
    df_hour_wise = df_hour_wise.round().astype(int)
    
    def categorize_time(hour_diff):
        categories = pd.Categorical(['< 12 hours', '12-24 hours', '24-48 hours', '> 48 hours'], ordered=True)
        if hour_diff > 48:
            return categories[3]  # '> 48 hours'
        elif hour_diff > 24 and hour_diff < 48:
            return categories[2]  # '24-48 hours'
        elif hour_diff > 11:
            return categories[1]  # '12-24 hours'
        else:
            return categories[0]  # '< 12 hours'
    
    dataframe["hour_diff"] = df_hour_wise
    dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
    data = dataframe[bucket_name].value_counts()
    return data


zo_ph_secondary_total = calculate_and_categorize_time(df_secondary_zo_ph , 'fact_updated_at' , current_time , "ZO Secondary Pending PH: ")
print(zo_ph_secondary_total)

zo_sph_secondary_total = calculate_and_categorize_time(df_secondary_zo_sph , 'fact_updated_at' , current_time , "ZO Secondary Pending SPH: ")
print(zo_sph_secondary_total)

print(f"Total ZO Secondary Pending:    {sum(zo_ph_secondary_total) + sum(zo_sph_secondary_total)}")


b5_ph_secondary_total = calculate_and_categorize_time(df_secondary_b5_ph , 'fact_updated_at' , current_time , "B5 Secondary Pending PH: ")
print(b5_ph_secondary_total)

b5_sph_secondary_total = calculate_and_categorize_time(df_secondary_b5_sph , 'fact_updated_at' , current_time , "B5 Secondary Pending SPH: ")
print(b5_sph_secondary_total)

print(f"Total B5 Secondary Pending:  {sum(b5_ph_secondary_total) + sum(b5_sph_secondary_total)} ")

ZO Secondary Pending PH: 
12-24 hours    896
> 48 hours     326
24-48 hours    174
Name: count, dtype: int64
ZO Secondary Pending SPH: 
12-24 hours    120
> 48 hours      75
24-48 hours     38
Name: count, dtype: int64
Total ZO Secondary Pending:    1629


/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["hour_diff"] = df_hour_wise
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

B5 Secondary Pending PH: 
12-24 hours    5490
24-48 hours     148
> 48 hours      142
Name: count, dtype: int64
B5 Secondary Pending SPH: 
12-24 hours    287
> 48 hours      39
24-48 hours     38
Name: count, dtype: int64
Total B5 Secondary Pending:  6144 


/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[bucket_name] = df_hour_wise.apply(categorize_time)
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["hour_diff"] = df_hour_wise
/var/folders/_4/bc29t88d5ml7g9p13zkltr6w0000gn/T/ipykernel_10284/2783421177.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic